In [3]:
import numpy as np
import numexpr as ne
import numba
import time

from tqdm.notebook import tqdm

In [14]:
print(f'{np.float64(1e-7):.50f}')

0.00000009999999999999999547481118258862586856139387


In [18]:
np.finfo(np.float16)

finfo(resolution=0.001, min=-6.55040e+04, max=6.55040e+04, dtype=float16)

In [46]:
M = int(1e3)
N = int(10e3)
SHAPE = (M, N)

a = np.ones(shape=SHAPE, dtype=np.complex128)
b = np.random.uniform(0, 1000, size=SHAPE).astype(np.float64)
c = np.ones(shape=SHAPE, dtype=np.float64)

print(f'a shape: {a.shape}, b shape: {b.shape}, c shape: {c.shape}')
print(f'a dtype: {a.dtype}, b dtype: {b.dtype}, c dtype: {c.dtype}')

a shape: (1000, 10000), b shape: (1000, 10000), c shape: (1000, 10000)
a dtype: complex128, b dtype: float64, c dtype: float64


In [47]:
# my_g[my_n] = self.data[n] * np.exp(1j * np.dot(self.Rg[n], q))

In [179]:
res = np.zeros(shape=M, dtype=np.complex128)

In [181]:
def f(x, y, z):
#     return x * np.exp(1j * np.dot(y, z))
    return np.exp(1j * np.dot(y, z))

In [191]:
t_start = time.time()
for idx, args in enumerate(zip(a, b, c)):
    res[idx] = f(*args)
print(f'took: {(time.time() - t_start) * 1000:.3f}ms')

took: 21.499ms


In [51]:
def step1(yy, zz):
    res = np.empty(len(yy), dtype=np.float64)
    for idx, (y, z) in enumerate(zip(yy, zz)):
        res[idx] = np.dot(y, z)
    return res

In [52]:
test = []
for x, y in zip(b, c):
    test.append(np.dot(x, y))

In [195]:
np.dot(a, res)

ValueError: shapes (1000,10000) and (1000,) not aligned: 10000 (dim 1) != 1000 (dim 0)

In [157]:
# @numba.jit(nopython=True)
def step1(xx, yy, zz):
    res = np.empty(shape=M, dtype=np.float64)
    for idx, (y, z) in enumerate(zip(yy, zz)):
        res[idx] = np.dot(y, z)
#         for i, j in zip(y, z):
#             _temp += i * j
#         res[idx] = _temp
    return res

In [158]:
step1(a,b,c)

array([4955460.85188843, 5013405.6145748 , 5029694.27719398,
       5019238.92712533, 5015128.35458856, 4953323.14766189,
       4988420.32118341, 5065140.33291555, 4997993.60996674,
       5002202.17576976, 5012518.96825993, 4948576.14703142,
       5038046.82899066, 4987118.88098776, 4942737.24425647,
       4991707.86391722, 5002077.32311893, 5009819.14746686,
       4992636.09221797, 4959983.11092125, 5006029.35609755,
       5017356.16432987, 5018789.07394197, 4982589.00116605,
       4986487.30237467, 4979116.480912  , 4953230.05727923,
       5058332.41082656, 5032303.5089066 , 5009492.82829641,
       5033698.65872317, 4973045.43994692, 4962370.73175642,
       5034244.41231178, 5020205.19255842, 5002360.39253517,
       5003251.44621916, 5008818.23989211, 5015951.54138167,
       5019938.86737378, 5032370.89445595, 5002376.33672291,
       5001359.8432811 , 5023245.73883344, 5022721.62408474,
       4976197.28773593, 4971132.85242041, 5034104.09114328,
       5014749.49940482,

In [159]:
%%time
step1(a,b,c)

Wall time: 15 ms


array([4955460.85188843, 5013405.6145748 , 5029694.27719398,
       5019238.92712533, 5015128.35458856, 4953323.14766189,
       4988420.32118341, 5065140.33291555, 4997993.60996674,
       5002202.17576976, 5012518.96825993, 4948576.14703142,
       5038046.82899066, 4987118.88098776, 4942737.24425647,
       4991707.86391722, 5002077.32311893, 5009819.14746686,
       4992636.09221797, 4959983.11092125, 5006029.35609755,
       5017356.16432987, 5018789.07394197, 4982589.00116605,
       4986487.30237467, 4979116.480912  , 4953230.05727923,
       5058332.41082656, 5032303.5089066 , 5009492.82829641,
       5033698.65872317, 4973045.43994692, 4962370.73175642,
       5034244.41231178, 5020205.19255842, 5002360.39253517,
       5003251.44621916, 5008818.23989211, 5015951.54138167,
       5019938.86737378, 5032370.89445595, 5002376.33672291,
       5001359.8432811 , 5023245.73883344, 5022721.62408474,
       4976197.28773593, 4971132.85242041, 5034104.09114328,
       5014749.49940482,